# Objectif : Comparer les produits structurés, effectuer du clustering pour identifier les outliers

Nous allons faire un embedding des descriptions de produits structurés ainsi que des caractéristiques que nous avons scrappé pour déterminer un score de similarité. Ensuite, nous effectuerons des algorithmes de clustering sur ce score pour déterminer qui sont les outliers, sous-entendu les produits structurés complexes.

Pour cela, il faudra faire apparaitre dans le résumé le détail du fonctionnement du produit structuré afin de pouvoir vraiment comparer.